In [1]:
import argparse
import os,sys
sys.path.append('/home/doyooni303/experiments/LLMRec/ReLLMRec')
os.chdir('/home/doyooni303/experiments/LLMRec/ReLLMRec')
import yaml

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AdamW
from peft import get_peft_model, LoraConfig, TaskType

from src.dataset.dataset import AmazonDataset

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--yaml', type=str, default='./configs/Books.yaml')
args = parser.parse_args([])
config = yaml.load(open(args.yaml, 'r'), Loader=yaml.FullLoader)

In [3]:
trainset = AmazonDataset(config, 'train')

splitting data by user: 100%|██████████| 1188598/1188598 [00:03<00:00, 344928.29it/s]


In [4]:
device = torch.device(f'cuda:{config["gpu"]}' if torch.cuda.is_available() else 'cpu')
os.environ['TOKENIZERS_PARALLELISM'] = 'true'
torch.random.manual_seed(config['seed'])
trainloader = DataLoader(trainset, batch_size=config['batch_size'], shuffle=True, num_workers=4)
batch = next(iter(trainloader))
for k,v in batch.items():
    if ("input_ids" in k) or ('attention_mask' in k):
        batch[k] = v.to(device)
    else:
        batch[k] = v
batch

{'user_id': tensor([393856,  71642]),
 'user_item_ids': ['347517 423591 164271', '110501 97412 101513 283686'],
 'ui_query_input_ids': tensor([[128000,   8586,    374,  ..., 128009, 128009, 128009],
         [128000,   8586,    374,  ..., 128009, 128009, 128009]],
        device='cuda:2'),
 'ui_query_attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]], device='cuda:2'),
 'su_query_input_ids': tensor([[128000,   8586,    374,  ..., 128009, 128009, 128009],
         [128000,   8586,    374,  ..., 128009, 128009, 128009]],
        device='cuda:2'),
 'su_query_attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]], device='cuda:2'),
 'target_item_id': tensor([91337, 85772]),
 'target_item_title': ['The Absolute Best Dump Cake Cookbook: More Than 60 Tasty Dump Cakes',
  "That's Not My Monster...(Usborne Touchy-Feely Books)"]}

In [5]:
# import torch
# import torch.nn.functional as F

# def find_top_k_item_embeddings(query_vectors, item_embeddings, device, batch_size=10000, k=5):
#     # Ensure inputs are on the correct device
#     if query_vectors.device != item_embeddings.device:
#         query_vectors = query_vectors.to(device)
#         item_embeddings = item_embeddings.to(device)
    
#     # Normalize the vectors for cosine similarity
#     query_vectors_normalized = F.normalize(query_vectors, p=2, dim=1)
#     item_embeddings_normalized = F.normalize(item_embeddings, p=2, dim=1)
    
#     # For large embedding matrices, compute similarity in batches
#     num_batches = (item_embeddings.shape[0] + batch_size - 1) // batch_size
    
#     # Initialize variables to store results on the GPU
#     num_queries = query_vectors.shape[0]
#     top_k_similarities = torch.full((num_queries, k), -float('inf'), device=device)
#     top_k_indices = torch.zeros((num_queries, k), dtype=torch.long, device=device)
    
#     # Compute similarities batch by batch
#     for i in range(num_batches):
#         start_idx = i * batch_size
#         end_idx = min((i + 1) * batch_size, item_embeddings.shape[0])
        
#         # Compute cosine similarity for this batch
#         batch_similarities = torch.matmul(
#             query_vectors_normalized, 
#             item_embeddings_normalized[start_idx:end_idx].t()
#         )
        
#         # If index 0 is in this batch, set its similarity to -inf to ignore it
#         if start_idx == 0 and end_idx > 0:
#             batch_similarities[:, 0] = -float('inf')
        
#         # For each query, get top-k similarities from this batch
#         batch_top_values, batch_top_indices = torch.topk(batch_similarities, min(k, batch_similarities.shape[1]), dim=1)
        
#         # Adjust indices to account for batching
#         batch_top_indices = batch_top_indices + start_idx
        
#         # Merge with existing top-k
#         if i == 0:
#             # For the first batch, just store the values
#             top_k_similarities[:, :batch_top_values.shape[1]] = batch_top_values
#             top_k_indices[:, :batch_top_indices.shape[1]] = batch_top_indices
#         else:
#             # For subsequent batches, merge and re-sort
#             combined_similarities = torch.cat([top_k_similarities, batch_top_values], dim=1)
#             combined_indices = torch.cat([top_k_indices, batch_top_indices], dim=1)
            
#             # Re-sort to get the overall top-k
#             for q in range(num_queries):
#                 q_top_values, q_top_indices = torch.topk(combined_similarities[q], k, dim=0)
#                 top_k_similarities[q] = q_top_values
#                 top_k_indices[q] = combined_indices[q][q_top_indices]
        
#         # Optional: Clear GPU cache periodically
#         if torch.cuda.is_available():
#             torch.cuda.empty_cache()
    
#     # Final check to ensure index 0 isn't in the results
#     for q in range(num_queries):
#         # Check if index 0 is in the results
#         zero_mask = top_k_indices[q] == 0
#         if torch.any(zero_mask):
#             # If found, replace with the next best item
#             remaining_indices = torch.ones(item_embeddings.shape[0], dtype=torch.bool, device=device)
#             # Mark indices already in top-k as used
#             remaining_indices[top_k_indices[q]] = False
#             # Also mark index 0 as used
#             remaining_indices[0] = False
            
#             # Find the count of indices to replace
#             num_to_replace = torch.sum(zero_mask).item()
            
#             if num_to_replace > 0 and torch.any(remaining_indices):
#                 # Find remaining embeddings
#                 remaining_embeddings = item_embeddings_normalized[remaining_indices]
                
#                 # Compute similarities for unused indices
#                 remaining_similarities = torch.matmul(
#                     query_vectors_normalized[q:q+1], 
#                     remaining_embeddings.t()
#                 )[0]
                
#                 # Get top replacements
#                 replacement_values, replacement_relative_indices = torch.topk(
#                     remaining_similarities, 
#                     min(num_to_replace, torch.sum(remaining_indices).item())
#                 )
                
#                 # Convert relative indices to absolute
#                 replacement_indices = torch.nonzero(remaining_indices, as_tuple=True)[0][replacement_relative_indices]
                
#                 # Replace zeros in the results
#                 zero_positions = torch.nonzero(zero_mask, as_tuple=True)[0]
#                 for j in range(min(len(zero_positions), len(replacement_indices))):
#                     top_k_indices[q, zero_positions[j]] = replacement_indices[j]
    
#     # Retrieve the actual embeddings using the indices
#     # Shape: [batch_size, k, embedding_dim]
#     top_k_item_embeddings = torch.zeros((num_queries, k, item_embeddings.shape[1]), device=device)
#     for q in range(num_queries):
#         top_k_item_embeddings[q] = item_embeddings[top_k_indices[q]]
    
#     return top_k_indices, top_k_item_embeddings

# # # Define your device
# # device = torch.device(f'cuda:1' if torch.cuda.is_available() else 'cpu')

# # # Find the top-5 most similar items and their embeddings
# # k = 5  # You specified k=5
# # top_k_indices, top_k_item_embeddings = find_top_k_item_embeddings(A, B, device, k=k)

# # # Print shapes to confirm
# # print(f"Shape of top_k_indices: {top_k_indices.shape}")           # Should be [4, 5]
# # print(f"Shape of top_k_item_embeddings: {top_k_item_embeddings.shape}")  # Should be [4, 5, 768]

In [6]:
# import random
# top_k = 5
# batch_size = 2
# dim = 768
# random.seed(config['seed'])
# torch.random.manual_seed(config['seed'])

# target_item_ids = torch.tensor([1,6,8,7])
# target_item_embeddings = torch.randn(batch_size, dim)

# top_k_indices = torch.tensor([random.sample(range(1,10), top_k) for _ in range(batch_size)])
# top_k_item_embeddings = torch.randn(batch_size, top_k, dim)

# perm_cols = torch.randperm(top_k)

# for i in range(config['batch_size']):
#     target = target_item_ids[i]
#     if target in top_k_indices[i]:
#         pass
#     else:
#         top_k_indices[i][-1] = target
# perm_k_indices = top_k_indices[:,perm_cols]
# perm_k_embeddings = top_k_item_embeddings[:, perm_cols, :]

# target_positions = torch.empty(config['batch_size'], dtype=torch.long)
# non_target_positions = torch.ones((config['batch_size'],config['top_k']), dtype=torch.bool)
# for i, (target_id, indices ) in enumerate(zip(target_item_ids, perm_k_indices)):
#     position = torch.where(target_id == indices)[0].item()
#     target_positions[i] = position
#     non_target_positions[i][position] = False
    
# linear = nn.Parameter(torch.randn(dim))
# output = torch.matmul(perm_k_embeddings, linear)

# ce_loss = F.cross_entropy(output, target_positions, reduction='mean')

In [7]:
import torch
import torch.nn.functional as F

class CandiRec(nn.Module):
    def __init__(self, config: dict, llm: AutoModelForCausalLM, device: torch.device, dtype: torch.dtype = torch.float16):
        super(CandiRec, self).__init__()
        torch.random.manual_seed(config['seed'])
        
        self.config = config
        self.llm = llm
        self.device = device
        self.dtype = dtype
        self.set_parameters()

    def set_parameters(self,):
        self.item_embeddings = torch.load(os.path.join(config['path'], f"{config['fname']}_item_embeddings.pt"))
        self.alignment_1 = nn.Linear(in_features=config['llm_dim'], out_features=config['latent_dim'])
        self.alignment_2 = nn.Linear(in_features=config['latent_dim'], out_features=config['item_dim'])
        self.alignment = nn.Sequential(self.alignment_1, self.alignment_2)
        self.linear = nn.Parameter(torch.randn(config['item_dim']))


    def find_top_k_item_embeddings(self, query_vectors, item_embeddings,batch_size=10000, k=5):
        # Ensure inputs are on the correct device
        if query_vectors.device != item_embeddings.device:
            query_vectors = query_vectors.to(device)
            item_embeddings = item_embeddings.to(device)
        
        # Normalize the vectors for cosine similarity
        query_vectors_normalized = F.normalize(query_vectors, p=2, dim=1).to(dtype=self.dtype)
        item_embeddings_normalized = F.normalize(item_embeddings, p=2, dim=1).to(dtype=self.dtype)

        
        # For large embedding matrices, compute similarity in batches
        num_batches = (item_embeddings.shape[0] + batch_size - 1) // batch_size
        
        # Initialize variables to store results on the GPU
        num_queries = query_vectors.shape[0]
        top_k_similarities = torch.full((num_queries, k), -float('inf'), device=device, dtype=self.dtype)
        top_k_indices = torch.zeros((num_queries, k),device= self.device, dtype=torch.long, )
        
        # Compute similarities batch by batch
        for i in range(num_batches):
            start_idx = i * batch_size
            end_idx = min((i + 1) * batch_size, item_embeddings.shape[0])
            
            # Compute cosine similarity for this batch
            batch_similarities = torch.matmul(
                query_vectors_normalized, 
                item_embeddings_normalized[start_idx:end_idx].t()
            )
            
            # If index 0 is in this batch, set its similarity to -inf to ignore it
            if start_idx == 0 and end_idx > 0:
                batch_similarities[:, 0] = -float('inf')
            
            # For each query, get top-k similarities from this batch
            batch_top_values, batch_top_indices = torch.topk(batch_similarities, min(k, batch_similarities.shape[1]), dim=1)
            
            # Adjust indices to account for batching
            batch_top_indices = batch_top_indices + start_idx
            
            # Merge with existing top-k
            if i == 0:
                # For the first batch, just store the values
                top_k_similarities[:, :batch_top_values.shape[1]] = batch_top_values
                top_k_indices[:, :batch_top_indices.shape[1]] = batch_top_indices
            else:
                # For subsequent batches, merge and re-sort
                combined_similarities = torch.cat([top_k_similarities, batch_top_values], dim=1)
                combined_indices = torch.cat([top_k_indices, batch_top_indices], dim=1)
                
                # Re-sort to get the overall top-k
                for q in range(num_queries):
                    q_top_values, q_top_indices = torch.topk(combined_similarities[q], k, dim=0)
                    top_k_similarities[q] = q_top_values
                    top_k_indices[q] = combined_indices[q][q_top_indices]
            
            # Optional: Clear GPU cache periodically
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
        
        # Final check to ensure index 0 isn't in the results
        for q in range(num_queries):
            # Check if index 0 is in the results
            zero_mask = top_k_indices[q] == 0
            if torch.any(zero_mask):
                # If found, replace with the next best item
                remaining_indices = torch.ones(item_embeddings.shape[0], dtype=torch.bool, device=self.device)
                # Mark indices already in top-k as used
                remaining_indices[top_k_indices[q]] = False
                # Also mark index 0 as used
                remaining_indices[0] = False
                
                # Find the count of indices to replace
                num_to_replace = torch.sum(zero_mask).item()
                
                if num_to_replace > 0 and torch.any(remaining_indices):
                    # Find remaining embeddings
                    remaining_embeddings = item_embeddings_normalized[remaining_indices]
                    
                    # Compute similarities for unused indices
                    remaining_similarities = torch.matmul(
                        query_vectors_normalized[q:q+1], 
                        remaining_embeddings.t()
                    )[0]
                    
                    # Get top replacements
                    replacement_values, replacement_relative_indices = torch.topk(
                        remaining_similarities, 
                        min(num_to_replace, torch.sum(remaining_indices).item())
                    )
                    
                    # Convert relative indices to absolute
                    replacement_indices = torch.nonzero(remaining_indices, as_tuple=True)[0][replacement_relative_indices]
                    
                    # Replace zeros in the results
                    zero_positions = torch.nonzero(zero_mask, as_tuple=True)[0]
                    for j in range(min(len(zero_positions), len(replacement_indices))):
                        top_k_indices[q, zero_positions[j]] = replacement_indices[j]
        
        # Retrieve the actual embeddings using the indices
        # Shape: [batch_size, k, embedding_dim]
        top_k_item_embeddings = torch.zeros((num_queries, k, item_embeddings.shape[1]), device=device, dtype=torch.float16)
        for q in range(num_queries):
            top_k_item_embeddings[q] = item_embeddings[top_k_indices[q]]
        
        return top_k_indices, top_k_item_embeddings
    
    
    def forward(self, batch: dict):
        torch.random.manual_seed(config['seed'])           

        # Target Item embeddings: [batch_size, item_dim]
        target_item_embeddings = self.item_embeddings[batch['target_item_id']]

        # User embeddings: [batch_size, item_dim]
        user_embeddings = torch.zeros(self.config['batch_size'], self.config['item_dim']).to(self.device)
        for i, ids in enumerate(batch['user_item_ids']):
            item_ids = [int(iid) for iid in ids.split(" ")]
            user_embeddings[i] += self.item_embeddings[item_ids].mean(dim=0)    

        # Get Query Vectors
        # output = self.llm(input_ids = batch['ui_query_input_ids'], attention_mask = batch['ui_query_attention_mask'])
        # print(output.logits.shape)
        
        item_history_query = self.llm(input_ids = batch['ui_query_input_ids'], attention_mask = batch['ui_query_attention_mask'], output_hidden_states=True).hidden_states[-1][:,-1,:]
        similar_user_query = self.llm(input_ids = batch['su_query_attention_mask'],attention_mask = batch['su_query_attention_mask'], output_hidden_states=True).hidden_states[-1][:,-1,:]
        
        query_embedding = self.alignment(item_history_query + similar_user_query)

        # Get Item Embeddings
        top_k_indices, top_k_item_embeddings = self.find_top_k_item_embeddings(query_embedding, self.item_embeddings, k=config['top_k'])
        # print(top_k_item_embeddings.shape, top_k_item_embeddings.dtype)
        perm_cols = torch.randperm(self.config['top_k'])
        for i, target in enumerate(batch['target_item_id']):
            if target in top_k_indices[i]:
                pass
            else:
                top_k_indices[i][-1] = target
        
        perm_k_indices = top_k_indices[:,perm_cols]
        # perm_target_item_id = batch['target_item_id'][perm_cols]
        perm_k_item_embeddings = top_k_item_embeddings[:, perm_cols, :]
        
        # Get Target Item positions
        target_positions = torch.empty(self.config['batch_size'], dtype=torch.long, device=self.device)
        non_target_positions = torch.ones((config['batch_size'],config['top_k']), dtype=torch.bool, device=self.device)
        for i, (target_id, indices ) in enumerate(zip(batch['target_item_id'], perm_k_indices)):
            position = torch.where(target_id == indices)[0].item()
            target_positions[i] = position
            non_target_positions[i][position] = False
        
        # Get Cross Entropy Loss
        # print(torch.matmul(perm_k_item_embeddings, self.linear).shape, torch.matmul(perm_k_item_embeddings, self.linear).dtype)
        # print(target_positions.shape, target_positions.dtype)
        ce_loss = F.cross_entropy(torch.matmul(perm_k_item_embeddings, self.linear), target_positions)

        # Get Contrastive loss
        negative_item_embeddings = perm_k_item_embeddings[non_target_positions].reshape(self.config['batch_size'], -1, self.config['item_dim']).transpose(1,2)
        cl_loss = (1-F.sigmoid(torch.bmm(target_item_embeddings.unsqueeze(1),negative_item_embeddings).squeeze(1))).mean(1)

        self.loss = ce_loss + cl_loss

        return top_k_indices, self.loss



        

In [9]:
device = torch.device(f'cuda:{config["gpu"]}' if torch.cuda.is_available() else 'cpu')
llm = AutoModelForCausalLM.from_pretrained(config['model_name'], torch_dtype=torch.float16, device_map=device)
lora_config = LoraConfig(task_type=getattr(TaskType, config['LoRA TaskType']),
                         **config['LoRA'])
llm = get_peft_model(llm, lora_config)
# model.print_trainable_parameters()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
model = CandiRec(config, llm, device).to(device=device, dtype=torch.float16)

In [11]:
model(batch)

IndexError: tensors used as indices must be long, int, byte or bool tensors